# Практика 8: Методы защиты от атак на модели ИИ
# Чурсинов Герман ББМО-01-23

## Цель задания:
Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных, моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак, изученных ранее.

## Задачи:
1. Изучить и реализовать защиту модели с помощью тренировок на противоречивых примерах (Adversarial Training).
2. Реализовать метод защиты на основе градиентной маскировки.
3. Использовать регуляризацию и нормализацию для повышения устойчивости модели.
4. Проверить эффективность методов защиты против атак FGSM, PGD и GAN-based атак.
5. Оценить улучшение точности защищенной модели на противоречивых примерах.

## Шаги выполнения:
**Шаг 1: Защита с помощью Adversarial Training**

Adversarial Training — это метод защиты, который заключается в том, чтобы обучать модель на противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам, так как она сталкивается с противоречивыми примерами на этапе обучения.




In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical

In [3]:
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images/255.0
test_images = test_images/255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  perturbation = epsilon * np.sign(gradient)
  adversarial_image = image + perturbation
  adversarial_image = np.clip(adversarial_image, 0, 1)
  return adversarial_image

In [4]:
# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
  adversarial_images = []
  for i in range(len(images)):
    image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
    label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)
    with tf.GradientTape() as tape:
      tape.watch(image)
      prediction = model(image)
      loss = tf.keras.losses.categorical_crossentropy(label, prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    adversarial_images.append(adversarial_image.reshape(28, 28))
  return np.array(adversarial_images)

In [5]:
# Создание модели
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
      ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
  for epoch in range(5): # Короткое обучение для демонстрации
    for i in range(0, len(train_images), 32):
      batch_images = train_images[i:i+32]
      batch_labels = train_labels[i:i+32]
      # Генерация противоречивых примеров для текущей партии данных
      adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
      # Объединение оригинальных и противоречивых примеров
      combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
      combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
      # Обучение на комбинированных данных
      model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()
# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)
# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


**Шаг 2: Градиентная маскировка (Gradient Masking)**

Gradient Masking — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он используется для уменьшения информации, доступной для атакующих, и усложнения поиска направленных изменений.

In [6]:
# Обновление модели для градиентной маскировки
def create_masked_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10),
      Activation('softplus') # Используем softplus вместо softmax для градиентной маскировки
      ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3900 - loss: 1.9965
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8249 - loss: 0.8474
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8919 - loss: 0.4558
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9354 - loss: 0.3030
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9391 - loss: 0.2563


**Шаг 3: Регуляризация и нормализация для повышения устойчивости**

Использование таких методов, как L2-регуляризация, дропаут и нормализация батчей, может помочь улучшить устойчивость модели к атакам.


In [7]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(10, activation='softmax')
      ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3002 - loss: 4.2247
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7192 - loss: 2.2067
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8053 - loss: 1.7316
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8538 - loss: 1.4756
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8884 - loss: 1.2810


**Шаг 4: Оценка моделей на противоречивых примерах**

Теперь проверим эффективность всех защитных методов на атакованных данных, созданных с помощью FGSM и других методов, таких как PGD или GAN.


In [8]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model, test_images, test_labels, epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4920 - loss: 1.4749
Accuracy of protected model on adversarial examples: 0.5473999977111816


Шаг 5: Сравнение методов защиты

Сравните эффективность различных методов защиты:
- Adversarial Training
- Gradient Masking
- Регуляризация и нормализация

Проанализируйте, какие методы наиболее эффективны для защиты от конкретных атак и какие комбинации дают наилучшие результаты.

Дополнительное задание (усложнение):
- Попробуйте комбинировать несколько методов защиты для повышения устойчивости модели.
- Оцените, как защищенные модели ведут себя на новых типах атак (например, GAN-based атаки).
- Исследуйте, как увеличенный уровень шума (epsilon) в атаке FGSM влияет на защищенные модели.

In [9]:
# Загрузка и подготовка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# --- Функция для создания модели ---
def create_model(activation='relu', regularization=None, use_batch_norm=False, dropout_rate=0.0):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.Dense(128, activation=activation, kernel_regularizer=regularization))
  if use_batch_norm:
    model.add(tf.keras.layers.BatchNormalization())
  if dropout_rate > 0.0:
    model.add(tf.keras.layers.Dropout(dropout_rate))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# --- Функция для FGSM атаки ---
def fgsm_attack(model, image, label, epsilon=0.1):
  image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))
  label = tf.convert_to_tensor(label.reshape((1, 10)))
  with tf.GradientTape() as tape:
    tape.watch(image)
    prediction = model(image)
    loss = tf.keras.losses.categorical_crossentropy(label, prediction)
  gradient = tape.gradient(loss, image)
  perturbed_image = image + epsilon * tf.sign(gradient)
  perturbed_image = tf.clip_by_value(perturbed_image, 0, 1)
  return perturbed_image.numpy()

# --- Функция для генерации FGSM противоречивых примеров ---
def generate_fgsm_adversarial_dataset(model, images, labels, epsilon=0.1):
  adversarial_images = []
  for i in range(len(images)):
    adversarial_image = fgsm_attack(model, images[i], labels[i], epsilon)
    adversarial_images.append(adversarial_image.reshape(28, 28))
  return np.array(adversarial_images)

# --- Adversarial Training (обучение на противоречивых примерах) ---
def adversarial_training(model, train_images, train_labels, epsilon=0.1, epochs=5, batch_size=32):
  for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    for i in range(0, len(train_images), batch_size):
      batch_images = train_images[i:i + batch_size]
      batch_labels = train_labels[i:i + batch_size]
      # Генерация противоречивых примеров
      adversarial_images = generate_fgsm_adversarial_dataset(model, batch_images, batch_labels, epsilon)
      # Комбинирование оригинальных и противоречивых данных
      combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
      combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
      # Обучение на комбинированных данных
      model.train_on_batch(combined_images, combined_labels)

# --- Реализация градиентной маскировки с помощью изменения функции активации ---
def create_masked_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128),
      tf.keras.layers.Activation('softplus'), # Использование softplus вместо relu для маскировки градиентов
      tf.keras.layers.Dense(10, activation='softmax')
      ])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# --- Основной код для эксперимента ---
# Создание и обучение базовой модели
base_model = create_model()
base_model.fit(train_images, train_labels, epochs=5)
# Создание и обучение модели с Adversarial Training
adv_model = create_model()
adversarial_training(adv_model, train_images, train_labels, epsilon=0.1)
# Создание и обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
# Создание модели с регуляризацией и дропаутом
regularized_model = create_model(regularization=tf.keras.regularizers.l2(0.01), use_batch_norm=True, dropout_rate=0.5)
regularized_model.fit(train_images, train_labels, epochs=5)

# --- Оценка моделей на противоречивых примерах ---
test_adversarial_images = generate_fgsm_adversarial_dataset(base_model,
test_images, test_labels, epsilon=0.1)
# Оценка защищенных моделей
print("Evaluation of models on adversarial examples:")
print("Base Model Accuracy on Adversarial Examples:")
base_model.evaluate(test_adversarial_images, test_labels)
print("Adversarially Trained Model Accuracy on Adversarial Examples:")
adv_model.evaluate(test_adversarial_images, test_labels)
print("Masked Model Accuracy on Adversarial Examples:")
masked_model.evaluate(test_adversarial_images, test_labels)
print("Regularized Model Accuracy on Adversarial Examples:")
regularized_model.evaluate(test_adversarial_images, test_labels)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4054 - loss: 1.9318
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8457 - loss: 0.7062
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8948 - loss: 0.4180
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9196 - loss: 0.3397
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9399 - loss: 0.2658
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3634 - loss: 2.0613
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8094 - loss: 0.8520
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8743 - loss: 0.5068
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9181 - loss: 0.3655
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9088 - loss: 0.3253
Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3292 - loss: 4.4686
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 

[2.2185733318328857, 0.5566999912261963]

# **Результаты:**

| Модель | Точность (FGSM) | Потери | Приемущества | Недостатки |
| :-- | :-: | :-: | :-: | --: |
| base_model	| 9.02% | 3.2060 | Простая структура | Полностью уязвима к атакам |
| adv_model |	59.00% | 1.1948 | Максимальная устойчивость | Высокая вычислительная стоимость обучения |
| masked_model |	38.75% | 1.9015 | Снижение доступа к градиентам | Защита менее эффективна для сложных атак |
| regularized_model | 50.86% | 2.3241 | Лучшая общая устойчивость	| Менее устойчива к FGSM по сравнению с Adversarial Training |

Наиболее эффективный метод защиты у `Adversarial Training`, так как он демонстрирует максимальную точность на противоречивых примерах (59%).

Gradient Masking и Regularization также повышают устойчивость, но их эффективность ниже, чем у Adversarial Training.

Base Model подтверждает свою полную уязвимость, что подчеркивает необходимость использования методов защиты.

# **Выводы:**

Для повышения точности и устойчивости к FGSM рекомендуется использовать `Adversarial Training`.

В ситуациях с ограниченными вычислительными ресурсами можно дополнительно рассмотреть сочетание регуляризации (L2 и Dropout) с Gradient Masking.